<a href="https://colab.research.google.com/github/ToiznerD/CloudComputing/blob/main/Project/user_interface_dor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [203]:
from zmq.constants import THREAD_AFFINITY_CPU_REMOVE
import json
from firebase import firebase
import IPython
from IPython.display import HTML
from google.colab import output, drive
from IPython.display import clear_output
from IPython.display import Markdown
import time
import ipywidgets as widgets
from google.colab.output import _publish as publish

def logo():
  display(IPython.display.HTML(''' <h1> Ipypy</h1> '''))



# Get style
drive.mount('/content/drive', force_remount=True)
with open('/content/drive/My Drive/MyStyles/style.css') as fid:
  style = fid.read()

FBconn = firebase.FirebaseApplication('https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/',None)
res=FBconn.get('/Questions/',None)
questions = []
rAnswers=0
wAnswers=0
qnum = 0
qnumLabel = None
questionLabel = None
container = None

################# LOGIN
def loginButton_eventhandler(obj):
  usersDB = FBconn.get('/Users/', None)
  
  # Check if user is exists in DB
  for key in usersDB:
      if usersDB[key]['username'] == username.value and usersDB[key]['password'] == password.value:
        label.value = 'You have successfully logged in!'
        time.sleep(2)
        clear_output(wait=True)
        loadGame()
        break
      else:
        label.value = 'Wrong details or user is not exists'

publish.css(style)
logo()

# User input
userLabel = widgets.Label(value='User Name:', layout=widgets.Layout(width='100px'))
userLabel.add_class('login')
username = widgets.Text(layout=widgets.Layout(width='250px'))
userHbox = widgets.HBox([userLabel, username])

# Password input
pwLabel = widgets.Label(value='Password:', layout=widgets.Layout(width='100px'))
pwLabel.add_class('login')
password = widgets.Text(layout=widgets.Layout(width='250px'))
password.add_class('login')
pwHbox = widgets.HBox([pwLabel, password])

# Login button
btn = widgets.Button(description='Login')
btn.on_click(loginButton_eventhandler)
btn.add_class('button')

# Error label
label = widgets.Label(value='', layout=widgets.Layout(height='100px'))
label.add_class('login')

container = widgets.VBox([userHbox, pwHbox, btn, label], layout=widgets.Layout(width='100%', align_items='center'))
display(container)



################# LOAD GAME
def loadGame():
  for key in res:
      questions.append(res[key])

  # Loading window

  publish.css(style)
  display(IPython.display.HTML('''
  <html>
    <head>
      <h1> Ipypy</h1>
      <h2>level 1</h2>
    </head>
    <body>

      <!-- Loading bar -->
      <div id="loading-bar">
        <div id="loading-progress"></div>
      </div>

      <!-- JavaScript code to simulate loading the game -->
      <script>
        var progress = 0;
        var loading = setInterval(function() {
          progress += Math.floor(Math.random() * 10); // simulate progress
          document.getElementById('loading-progress').style.width = progress + '%';
          if (progress >= 100) {
            clearInterval(loading); // stop the interval when progress reaches 100%
          }
        }, 500);
      </script>
    </body>
  </html>
  '''))

  time.sleep(2)

  # Clear loading window
  clear_output(wait=True)
  startGame()


################# START GAME
def startGame():

  ## UTILS ##

  def checkAnswer(ans, buttons):
    cButton = None
    global qnum

    # Check if container is visible
    if container.layout.display == 'none':
      return

    # Find the correct answer
    for b in buttons:
      if b.description == questions[qnum]['correct']:
        cButton = b
    
    # Validate the clicked answer
    if ans.description == cButton.description:
      ans.add_class('correctAnswer')
      global rAnswers
      rAnswers += 1
    else:
      ans.add_class('wrongAnswer')
      cButton.add_class('correctAnswer')
      global wAnswers
      wAnswers += 1
    
    time.sleep(3)
    
    qnum += 1
    if qnum == len(questions): # in case of last question
      container.layout.display = 'none'
      rAnswersLabel = widgets.HTML(value=f'Correct Answers: {rAnswers}')
      rAnswersLabel.add_class('h5')
      wAnswersLabel = widgets.HTML(value=f'Wrong Answers: {wAnswers}')
      wAnswersLabel.add_class('h4')
      answersContainer = widgets.VBox([rAnswersLabel, wAnswersLabel], layout=widgets.Layout(align_items='center', height="250px"))
      display(answersContainer)
      return
    
    # Present next question
    questionLabel.value = questions[qnum]['question']
    qnumLabel.value = f'Question #{qnum+1}'
    for i,b in enumerate(buttons):
      ansStr = ansStr = f'ans{i+1}'
      b.description = questions[qnum][ansStr]
      b.remove_class("wrongAnswer")
      b.remove_class("correctAnswer")
      b.add_class("button")
  
  ## END UTILS ##

  publish.css(style)

  # Present logo
  logo()

  button_layout = {'width': '350px','height': '100px', 'margin': '10px'}
  buttons = []
  for i in range(1,5):
    ansStr = f'ans{i}'
    b = widgets.Button(description=questions[qnum][ansStr], layout=button_layout)
    b.add_class("button")
    buttons.append(b)

  button_container = widgets.HBox(children=buttons)

  # Add listeners
  for b in buttons:
    b.on_click(lambda button: checkAnswer(button, buttons))


  # Wrap the button container in a VBox container and center it horizontally
  qnumLabel = widgets.HTML(value=f'Question #{qnum+1}')
  qnumLabel.add_class("h2")
  questionLabel = widgets.HTML(value=questions[0]['question'])
  questionLabel.add_class("h2")

  container = widgets.VBox([qnumLabel, questionLabel, button_container], layout=widgets.Layout(align_items='center'))
  display(container)


Mounted at /content/drive


In [ ]:
from firebase import firebase
from google.colab import drive
import ipywidgets as widgets
from google.colab.output import _publish as publish


FBconn = firebase.FirebaseApplication('https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/', None)

def loginButton_eventhandler(obj):
  usersDB = FBconn.get('/Users/', None)
  
  # Check if user is exists in DB
  for key in usersDB:
      if usersDB[key]['username'] == username.value and usersDB[key]['password'] == password.value:
        label.value = 'You have successfully logged in!'
        break
      else:
        label.value = 'Wrong details or user is not exists'


#@title LoginForm

# Get style
drive.mount('/content/drive', force_remount=True)
with open('/content/drive/My Drive/MyStyles/style.css') as fid:
  style = fid.read()

username = widgets.Text(description="User Name:", layout=widgets.Layout(width='250px'))

password = widgets.Password(description="Password:", layout=widgets.Layout(width='250px'))

btn = widgets.Button(description='Login')
btn.on_click(loginButton_eventhandler)
btn.add_class('button')

label = widgets.Label(value='', layout=widgets.Layout(height='100px'))
publish.css(style)

display(username, password, btn, label)




In [149]:
from firebase import firebase
from google.colab import drive
import ipywidgets as widgets
from google.colab.output import _publish as publish


FBconn = firebase.FirebaseApplication('https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/', None)
users_to_upload = {
    'Username': 'dor',
    'Password': 'dor'
}
FBconn.post('/Users/', users_to_upload)

{'name': '-NVQ4A4bR66Y1hEy1Gib'}

In [198]:
publish.css(style)
userLabel = widgets.Label(value='User Name:', layout=widgets.Layout(width='100px'))
userLabel.add_class('login')
username = widgets.Text(layout=widgets.Layout(width='250px'))


pwLabel = widgets.Label(value='Password:', layout=widgets.Layout(width='100px'))
pwLabel.add_class('login')
password = widgets.Text(layout=widgets.Layout(width='250px'))
password.add_class('login')


userHbox = widgets.HBox([userLabel, username])
pwHbox = widgets.HBox([pwLabel, password])
container = widgets.VBox([userHbox, pwHbox], layout=widgets.Layout(width='100%', align_items='center'))
display(container)
